<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **spacex_dash_app**


In [3]:
!pip install dash

In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px
import numpy as np


/var/folders/l9/78bm0d1j7qj5p8wrxdjn3d5h0000gn/T/ipykernel_62067/2517657126.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/l9/78bm0d1j7qj5p8wrxdjn3d5h0000gn/T/ipykernel_62067/2517657126.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [5]:
# Read the spacex_launch_dash.csv file
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# spacex_df.head()

In [6]:
# for TASK 1
spacex_df_aux = spacex_df[['Launch Site']]
launch_sites_df = spacex_df_aux.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site']]
#launch_sites_df

In [7]:
# Create a dash application
app = dash.Dash(__name__)

In [8]:
# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [9]:
# options_sites_list = [{'label': i, 'value': i} for i in zip(launch_sites_df['Launch Site'])]
sites_list = [{'label': i, 'value': i} for i in launch_sites_df['Launch Site']]

# sites_list

In [10]:
#options_sites
options_dropdown = [{'label': 'All Sites', 'value': 'ALL'}] + sites_list
# options_dropdown

In [11]:
# myColor_pie = '#D0D5E8'
launch_site = "ALL"

In [12]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                 dcc.Dropdown( #id='site-dropdown',
                                                options = options_dropdown,
                                                placeholder='Select a launch site',
                                                style={'width':'100%', 'padding':'3px', 'font-size': '15px', 'text-align-last' : 'left'},
                                                id='site-dropdown',
                                                ),
                                html.Br(),
                                html.Br(),
                                
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div([html.H2("Success pie chart")], id='success-pie-chart'),
#                                html.Div(dcc.Graph(id='success-pie-chart')),
#                                                    html.H2('Success pie chart'),
#                                html.Div([html.H2("Sucess pie chart")], style={'textAlign': 'center', 'background': myColor_pie, 'width': '50%', 'border-style': 'ridge'}, id='success-pie-chart'),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    # marks={0: '0', 100: '100'},
                                    value=[min_payload, max_payload], allowCross=False),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
#                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Div(id='success-payload-scatter-chart'),
                                ])


                                

In [13]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
#@app.callback( [Output(component_id='success-pie-chart', component_property='children')],

'''     
# version 2
# cuando se pide por sitio, se muestra Success y Failed
# cuando se pide por "ALL", se muestran los Success de cada sitio
# if launch_site = 'ALL'
expected df_aux shape
############################
     Launch Site  Success
0   CCAFS LC-40        7
1  CCAFS SLC-40        3
2    KSC LC-39A       10
3   VAFB SLC-4E        4
############################
else
the name of the 1st column corresponds to the selected site
############################
   CCAFS LC-40  Count
0     Success      7
1      Failed     19
############################
or 
############################
   VAFB SLC-4E  Count
0     Success      4
1      Failed      6
############################
or whatever, depending on the 'launch_site' value
'''


@app.callback( Output(component_id='success-pie-chart', component_property='children'),
               Input(component_id='site-dropdown', component_property='value')
              )

def update_pie(launch_site):

        # Select data
        if launch_site == 'ALL':
            df_aux =  spacex_df[['Launch Site', 'class']].copy()
            # aprovechamos que "class" es "success = 1" y "failed = 0", entonces podemos sumar los "class" y nos da
            # la cantidad de lanzamientos ok. 
            # .sum() = lanzamientos ok
            # .count() = total de lanzamientos
            # .count() - .sum() = lanzamientos failed
            df_aux = pd.DataFrame(df_aux.groupby(['Launch Site'],as_index=False)['class'].sum())
            df_aux.columns=['Launch Site', 'Success']
            
            # parameters for pie
            values = 'Success'
            names = 'Launch Site'
            color = 'Launch Site'
            color_discrete_map=None # default colors
            title='Count of successful launches in all sites: ' + str(df_aux['Success'].sum())
        else:
            aux_ok = spacex_df[(spacex_df['Launch Site']==launch_site) & (spacex_df['class'] == 1)]['class'].count()
            aux_failed = spacex_df[(spacex_df['Launch Site']==launch_site) & (spacex_df['class'] == 0)]['class'].count()
            df_aux =  pd.DataFrame([['Success', aux_ok], ['Failed', aux_failed]], columns=['Result', 'Count'])
            df_aux.columns=[launch_site, 'Count']
            # parameters for pie
            values = 'Count'
            names = launch_site #name of the column that will take values Success or Failed
            color = launch_site #name of the column that will take values Success or Failed
            color_discrete_map={'Success':'green', 'Failed':'red'}
            title='Count of Successful vs Failed launches in selected site: ' + launch_site + '<br>  Total launches: ' + str(aux_ok+aux_failed) + '<br>  Successful launches: ' + str(aux_ok) + '<br>  Unsuccessful launches: ' + str(aux_failed)

        pie_fig = px.pie(df_aux, values=values, names=names, color=color,
             color_discrete_map=color_discrete_map, title=title)
        
        return(dcc.Graph(figure=pie_fig))


In [14]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback( Output(component_id='success-payload-scatter-chart', component_property='children'),
               [Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id="payload-slider", component_property="value")]
              )

def update_scattered(launch_site, payload_sldr):
    
    aux_min_PLM = payload_sldr[0]
    aux_max_PLM = payload_sldr[1]

    # filtering spacex_df with payload mass in range
    df_aux =  spacex_df[(spacex_df['Payload Mass (kg)'] >= aux_min_PLM)
                        & (spacex_df['Payload Mass (kg)'] <= aux_max_PLM)]
    cc = -1

    # parameters for scatter
    x = 'Payload Mass (kg)'
    y = 'class'
    color = 'Booster Version Category'
    symbol = 'Booster Version Category'
    
    if launch_site == 'ALL': 
#        df_aux has been set before the if for ALL values in range of payload mass as default
        title='Payload detail for all sites with payload range: '
    else:
# filtering df_aux for a specific site        
        df_aux =  df_aux[(df_aux['Launch Site']==launch_site)]
        title='Payload detail for site: ' + launch_site + ', with payload range: '

    cc = df_aux['class'].count()
    cc_ok = df_aux[df_aux['class'] == 1]['class'].count()
    cc_failed = df_aux[df_aux['class'] == 0]['class'].count()
    title = title + str(aux_min_PLM) + ' - ' + str(aux_max_PLM) + ' - ' + str(cc) + ' launches / ' + str(cc_ok) + ' Ok / ' + str(cc_failed) + ' failed.'
    scatter_fig = px.scatter(df_aux, x=x, y=y, color=color, symbol=symbol, title=title)
    return(dcc.Graph(figure=scatter_fig))


In [15]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)


[2024-11-13 17:01:18,596] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.12/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/lib/python3.12/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/lib/python3.12/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/lib/python3.12/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/lib/python3.12/site-